In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Download dependencies

In [ ]:
!pip install --upgrade transformers

     |████████████████████████████████| 3.1 MB 4.2 MB/s 
     |████████████████████████████████| 895 kB 62.0 MB/s 
     |████████████████████████████████| 596 kB 65.7 MB/s 
     |████████████████████████████████| 3.3 MB 55.0 MB/s 
     |████████████████████████████████| 56 kB 5.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install pytorch-transformers

     |████████████████████████████████| 176 kB 4.2 MB/s 
     |████████████████████████████████| 131 kB 56.9 MB/s 
     |████████████████████████████████| 1.2 MB 65.8 MB/s 
     |████████████████████████████████| 8.1 MB 35.6 MB/s 
     |████████████████████████████████| 79 kB 10.2 MB/s 
     |████████████████████████████████| 138 kB 77.6 MB/s 
     |████████████████████████████████| 127 kB 75.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
from transformers import __version__
print(__version__)

4.12.0


In [ ]:
!pip install sentencepiece

In [ ]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 4.2 MB/s 
     |████████████████████████████████| 180 kB 69.4 MB/s 
     |████████████████████████████████| 139 kB 61.0 MB/s 
     |████████████████████████████████| 97 kB 8.9 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=7167d3a425577760297ea2d21848d630410694ee1a144a9e2110d5c7cab6f88b
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=dda6b755de2fb949cdde95aa508900d590752497e0b096c3d383df9bccecb216
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


### Import libraries

In [ ]:
import pandas as pd
import numpy as np
import re
import regex
from tqdm import tqdm

from sklearn import preprocessing, metrics
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve

import torch
import torch.nn.functional as F
from torch import nn, optim
from torch.utils.data import TensorDataset,RandomSampler,SequentialSampler
from torch.utils.data import Dataset, DataLoader

from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

import tensorflow as tf
import transformers
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import RobertaModel, RobertaTokenizer

import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import WordPunctTokenizer
from string import punctuation
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
print(torch.__version__)

1.9.0+cu111


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

### Load data

In [ ]:
f_path = '/content/drive/MyDrive/3.1/CZ4042 Neural Networks & Deep Learning/Project/emotion/dataset/crowdflower_text_emotion.csv'

In [ ]:
df = pd.read_csv(f_path)

### Data preprocessing and analysis

In [ ]:
df.head()

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...


In [ ]:
# Check emotions labels

labels = df.sentiment.unique()
print(labels)

['empty' 'sadness' 'enthusiasm' 'neutral' 'worry' 'surprise' 'love' 'fun'
 'hate' 'happiness' 'boredom' 'relief' 'anger']


In [ ]:
# Remove irrelevant columns
df = df.drop(columns=['tweet_id', 'author'])

In [ ]:
# Check for null cells
df['sentiment'].isnull().values.any()


False

In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()

stop = stopwords.words('english')

for punct in punctuation:
    stop.append(punct)

def clean(x, stop_words):
    word_tokens = WordPunctTokenizer().tokenize(x.lower())
    x = [regex.sub(u'\p{^Latin}', u'', w) for w in word_tokens if w.isalpha() and len(w) > 2]
    return " ".join(x)

df["content_cleaned"] = df.content.apply(lambda x : clean(x, stop)) 

In [ ]:
# Check len of each content
df['content_count'] = df['content_cleaned'].str.split(" ").str.len()

In [ ]:
df.head()

,sentiment,content,content_cleaned,content_count
0,empty,@tiffanylue i know i was listenin to bad habi...,tiffanylue know was listenin bad habit earlier...,12
1,sadness,Layin n bed with a headache ughhhh...waitin o...,layin bed with headache ughhhh waitin your call,8
2,sadness,Funeral ceremony...gloomy friday...,funeral ceremony gloomy friday,4
3,enthusiasm,wants to hang out with friends SOON!,wants hang out with friends soon,6
4,neutral,@dannycastillo We want to trade with someone w...,dannycastillo want trade with someone who has ...,12


### Label encoding classes

In [ ]:
labels_dict = dict(zip(labels, (x for x in range(len(labels)) )))
df['sentiment_val'] = df['sentiment'].map(labels_dict)

In [ ]:
df = df.drop(columns=['sentiment', 'content_count', 'content'])

In [ ]:
df.head()

,content_cleaned,sentiment_val
0,tiffanylue know was listenin bad habit earlier...,0
1,layin bed with headache ughhhh waitin your call,1
2,funeral ceremony gloomy friday,1
3,wants hang out with friends soon,2
4,dannycastillo want trade with someone who has ...,3


### Prep data for training/validation

In [ ]:
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.content_cleaned
        self.targets = self.data.sentiment_val
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
# Batch Size
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 2
                }

val_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 2
                }

### RoBERTa models

#### Base model

In [ ]:
# Initialize tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 13)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

### Train/Eval functions

#### Parameters

In [ ]:
LEARNING_RATE = 1e-05
MAX_LEN = 256

# Multi class loss
loss_function = torch.nn.CrossEntropyLoss()

# Accuracy function
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

#### Training

In [ ]:
# For RoBERTa base
def train(model, epoch, training_loader):

  tr_loss = 0
  n_correct = 0
  nb_tr_steps = 0
  nb_tr_examples = 0

  # Tell wandb to watch this model train
  wandb.watch(model)

  model.train()

  for _, data in tqdm(enumerate(training_loader, 0)):

    ids = data['ids'].to(device, dtype = torch.long)
    mask = data['mask'].to(device, dtype = torch.long)
    token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
    targets = data['targets'].to(device, dtype = torch.long)

    outputs = model(ids, mask, token_type_ids)
    loss = loss_function(outputs, targets)
    tr_loss += loss.item()
    big_val, big_idx = torch.max(outputs.data, dim=1)
    n_correct += calcuate_accuracy(big_idx, targets)

    nb_tr_steps += 1
    nb_tr_examples+=targets.size(0)
    
    if _%5000==0:
        loss_step = tr_loss/nb_tr_steps
        accu_step = (n_correct*100)/nb_tr_examples 

    optimizer.zero_grad()
    loss.backward()
    # Optimizer 
    optimizer.step()

  print('\n')
  epoch_loss = tr_loss/nb_tr_steps
  epoch_accu = (n_correct*100)/nb_tr_examples
  print(f"Training Loss Epoch {epoch+1}: {epoch_loss}")
  print(f"Training Accuracy Epoch {epoch+1}: {epoch_accu} \n")
  
  return epoch_accu, epoch_loss

#### Eval

In [ ]:
def valid(model, testing_loader):

    n_correct = 0; 
    n_wrong = 0; 
    total = 0; 
    tr_loss=0; 
    nb_tr_steps=0; 
    nb_tr_examples=0

    model.eval()

    with torch.no_grad():

        for _, data in tqdm(enumerate(testing_loader, 0)):

            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accuracy(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
        # Reduce LR on val loss plateau
        scheduler.step(loss)

    print('\n')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch : {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    
    return epoch_accu, epoch_loss


### K-Fold (RoBERTa base)

In [ ]:
# Set K-fold
kf = KFold(n_splits=3, shuffle=True, random_state=42)

In [ ]:
# # Setup wandb for logging
# import wandb

# wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
EPOCHS=15
f = 1
df = df.reset_index(drop=True)

for train_index, val_index in kf.split(df):

  # Init model
  model = RobertaClass()
  model.to(device)

  # Optimizer
  optimizer = torch.optim.AdamW(params=model.parameters(), lr=LEARNING_RATE)
  # Reduce LR on plateau
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

  # Init wandb for logging
  # wandb.init(project="Text_Emotion_Recognition_RoBERTa", name=f"3-Fold")

  train2 = df.iloc[train_index]
  val2 = df.iloc[val_index]

  train2 = train2.reset_index(drop=True)
  val2 = df.drop(train2.index).reset_index(drop=True)

  training_set = SentimentData(train2, tokenizer, MAX_LEN)
  val_set = SentimentData(val2, tokenizer, MAX_LEN)

  training_loader = DataLoader(training_set, **train_params)
  val_loader = DataLoader(val_set, **val_params)

  print(f"Training fold {f}...")

  for epoch in range(0, EPOCHS):
    
    # Train
    train_acc, train_loss = train(model, epoch, training_loader)
    # Val
    val_acc, val_loss = valid(model, val_loader)
    print(f"Epoch {epoch+1} done!")
    # Log metrics
    # wandb.log({"Train_Loss": train_loss, "Val_Loss": val_loss, "Train_Acc": train_acc, "Val_Acc": val_acc, "Epoch": epoch})   

  f+=1
  # wandb.finish()

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
wandb: Currently logged in as: wasabee (use `wandb login --relogin` to force relogin)


Training fold 1...


0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
3334it [12:23,  4.49it/s]



Training Loss Epoch 1: 1.9214037464418738
Training Accuracy Epoch 1: 34.57961449036226 



...



Validation Loss Epoch : 1.2084762191963643
Validation Accuracy Epoch: 75.67689192229805
Epoch 14 done!


0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
3334it [12:46,  4.35it/s]



Training Loss Epoch 15: 0.32960516793612027
Training Accuracy Epoch 15: 89.19638504518693 




0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
1667it [02:00, 13.84it/s]




Validation Loss Epoch : 1.261165333744172
Validation Accuracy Epoch: 75.27938198454962
Epoch 15 done!


### Save model

In [ ]:
# Base model
rob_base = model

torch.save(rob_base, '/content/drive/MyDrive/3.1/CZ4042 Neural Networks & Deep Learning/Project/emotion/models/roberta_base/roberta_base.bin')
tokenizer.save_vocabulary('/content/drive/MyDrive/3.1/CZ4042 Neural Networks & Deep Learning/Project/emotion/models/roberta_base/')

('/content/drive/MyDrive/3.1/CZ4042 Neural Networks & Deep Learning/Project/emotion/models/roberta_base/vocab.json',
 '/content/drive/MyDrive/3.1/CZ4042 Neural Networks & Deep Learning/Project/emotion/models/roberta_base/merges.txt')

### Inference

#### Load model, tokenizer, labels

In [ ]:
!cp '/content/drive/MyDrive/3.1/CZ4042 Neural Networks & Deep Learning/Project/emotion/models/roberta_sent.bin' -d '/content/roberta_sent.bin'

In [ ]:
!cp '/content/drive/MyDrive/3.1/CZ4042 Neural Networks & Deep Learning/Project/emotion/models/vocab.json' -d '/content/vocab.json'

In [ ]:
!cp '/content/drive/MyDrive/3.1/CZ4042 Neural Networks & Deep Learning/Project/emotion/models/merges.txt' -d '/content/merges.txt'

In [ ]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("/content/roberta_sent.bin")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 13)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [ ]:
saved_model2 = torch.load('/content/roberta_sent.bin')

In [ ]:
tokenizer2 = RobertaTokenizer.from_pretrained("/content/")

file /content/config.json not found


In [ ]:
labels_dict = {'anger': 12,
 'boredom': 10,
 'empty': 0,
 'enthusiasm': 2,
 'fun': 7,
 'happiness': 9,
 'hate': 8,
 'love': 6,
 'neutral': 3,
 'relief': 11,
 'sadness': 1,
 'surprise': 5,
 'worry': 4}

#### Function to get encode user input

In [ ]:
def get_inputs(text, tokenizer):

  inps = tokenizer.encode_plus(
              list(text),
              None,
              add_special_tokens=True,
              max_length=256,
              pad_to_max_length=True,
              return_token_type_ids=True,
              return_tensors='pt'
          )
  
  inp_tok = inps['input_ids'].to('cuda', dtype = torch.long)
  ids = inps['attention_mask'].to('cuda', dtype = torch.long)
  segments = inps['token_type_ids'].to('cuda', dtype = torch.long)

  return inp_tok, ids, segments


In [ ]:
def cleaner(user_text):

  wordnet_lemmatizer = WordNetLemmatizer()

  stop = stopwords.words('english')

  for punct in punctuation:
      stop.append(punct)

  def clean(x, stop_words):
      word_tokens = WordPunctTokenizer().tokenize(x.lower())
      x = [regex.sub(u'\p{^Latin}', u'', w) for w in word_tokens if w.isalpha() and len(w) > 2]
      return " ".join(x)

  cleaned_user = clean(user_text, stop) 
  
  return cleaned_user

In [ ]:
def get_sentiment(user_input):

  # Define empty list to store emotions
  sentiment = []

  user_input = cleaner(user_input)

  # Pass user input to encode text
  inp_tok, ids, segments = get_inputs(user_input, tokenizer2)

  # Get predictions
  preds = saved_model2(inp_tok, ids, segments)

  # Get max pred
  # big_val, big_idx = torch.max(preds.data, dim=1)
  big_val, big_idx = torch.topk(preds.data, k=3, dim=1)

  for x in range(len(big_idx[0])):
    sentiment.append(list(labels_dict.keys())[list(labels_dict.values()).index(big_idx[0][x].item())])

  print("Predicted Sentiment:", sentiment)

In [ ]:
# write to get user input here
# user_input = input("Enter some text: ")
user_input = 'missed the bus.... shit!'

In [ ]:
get_sentiment(user_input)

Predicted Sentiment: ['sadness', 'neutral', 'worry']


In [ ]:
labels_dict

{'anger': 12,
 'boredom': 10,
 'empty': 0,
 'enthusiasm': 2,
 'fun': 7,
 'happiness': 9,
 'hate': 8,
 'love': 6,
 'neutral': 3,
 'relief': 11,
 'sadness': 1,
 'surprise': 5,
 'worry': 4}

In [ ]:
from random import randint

r_int = randint(0,100)

print(df.sentiment[r_int])
print(df.content_cleaned[r_int])

neutral
missed the bus
